In [1]:
import modelseedpy
from modelseedpy import MSGenome, MSATPCorrection, MSGapfill, MSBuilder, RastClient, MSMedia
from modelseedpy.fbapkg.mspackagemanager import MSPackageManager
modelseedpy.core.msatpcorrection.logger.setLevel('DEBUG')
modelseedpy.core.msgapfill.logger.setLevel('DEBUG')

In [2]:
rast = RastClient()

In [3]:
genome = MSGenome.from_fasta('./GCF_000005845.2_ASM584v2_protein.faa', split=' ')

In [4]:
res = rast.annotate_genome(genome)

In [5]:
from modelseedpy.helpers import get_template
from modelseedpy.core.mstemplate import MSTemplateBuilder

In [6]:
template = get_template('template_core')
template = MSTemplateBuilder.from_dict(template).build()

In [56]:
b = MSBuilder(genome, template)

In [57]:
model = b.build('model', '0', False, False)
model.objective = 'bio2'

In [9]:
import pandas as pd
def load_medias(filep):
    medias = pd.read_csv(filep, sep='\t', index_col=0).to_dict()
    return medias
medias = load_medias('core_medias.tsv')
model_medias = {}
model_exs = set(map(lambda x: x.id, model.exchanges))
for media_id in medias:
    media = medias[media_id]
    media_values = dict(map(lambda x: (x[0][3:][:-3], x[1]), filter(lambda x: x[1] > 0, media.items())))
    media = MSMedia.from_dict(media_values)
    media.id = media_id
    model_medias[media_id] = media

In [10]:
media = model_medias['Ac']
media.get_media_constraints()

{'cpd00001_e0': (-1000, 1000),
 'cpd00067_e0': (-1000, 1000),
 'cpd00009_e0': (-1000, 1000),
 'cpd00013_e0': (-1000, 1000),
 'cpd00029_e0': (-3, 1000),
 'cpd00011_e0': (-1000, 1000)}

In [37]:
from modelseedpy.fbapkg.mspackagemanager import MSPackageManager
pkm = MSPackageManager.get_pkg_mgr(model)

In [39]:
pkg_gapfill = pkm.getpkg("GapfillingPkg")

In [40]:
pkg_gapfill.compute_gapfilled_solution()

SolverError: CPLEX Error  1217: No solution exists.

In [35]:
atp_correction.msgapfill.compute_gapfilled_solution

AttributeError: 'MSGapfill' object has no attribute 'compute_gapfilled_solution'

In [31]:
atp_correction.msgapfill.run_gapfilling(media, 'bio2')

/usr/local/lib/python3.8/site-packages/cobra/util/solver.py:508 UserWarning: Solver status is 'infeasible'.


{'reversed': {}, 'new': {}}

In [33]:
atp_correction.msgapfill.solutions[media]

{'bio2': {'reversed': {}, 'new': {}}}

In [11]:
import cobra
model_exs = set(map(lambda x: x.id, model.exchanges))

for media_id in medias:
    media = medias[media_id]
    media_values = dict(filter(lambda x: x[0] in model_exs and x[1] > 0, media.items()))
    model.medium = media_values
    sol = cobra.flux_analysis.pfba(model)
    print(media_id, sol.fluxes['bio2'])

Glc/O2 0.5516512859637026
Ac/O2 0.0
Etho/O2 0.0
Pyr/O2 0.0
Glyc/O2 0.6162062236828592
Fum/O2 0.31690605789404186
Succ/O2 0.0
Akg/O2 0.0
LLac/O2 0.0
Dlac/O2 0.0
For/O2 0.0
Glc 0.0
Ac 0.0
Etho 0.0
Pyr 0.0
Glyc 0.0
Fum 0.0
Succ 0.0
Akg 0.0
For/NO2 0.0
For/NO3 0.0
Glc/DMSO 0.0
Glc/TMAO 0.0
Pyr/DMSO 0.0
Pyr/TMAO 0.0
Pyr/NO 0.0
Pyr/NO2 0.0
Pyr/NO3 0.0
Pyr/SO4 0.0
Pyr/SO3 0.0
H2/CO2 0.0
H2/O2 0.0
empty 0.0
Light 0.0


In [15]:
sol = model.optimize()
print(sol.objective_value, sol.status)

0.0 optimal


In [22]:
for media in atp_correction.selected_media:
    if media in atp_correction.media_gapfill_stats and atp_correction.media_gapfill_stats[media]:
        stats = atp_correction.media_gapfill_stats[media]
        print(media.id, stats)

Ac/O2 {'reversed': {}, 'new': {}}
Etho/O2 {'reversed': {}, 'new': {}}
Pyr/O2 {'reversed': {}, 'new': {}}
Succ/O2 {'reversed': {}, 'new': {}}
Akg/O2 {'reversed': {}, 'new': {}}
Dlac/O2 {'reversed': {}, 'new': {}}
For/O2 {'reversed': {}, 'new': {}}
Ac {'reversed': {}, 'new': {}}
Etho {'reversed': {}, 'new': {}}
Pyr {'reversed': {}, 'new': {}}
Succ {'reversed': {}, 'new': {}}
Akg {'reversed': {}, 'new': {}}
For/NO2 {'reversed': {}, 'new': {}}
For/NO3 {'reversed': {}, 'new': {}}
Pyr/DMSO {'reversed': {}, 'new': {}}
Pyr/TMAO {'reversed': {}, 'new': {}}
Pyr/NO {'reversed': {}, 'new': {}}
Pyr/NO2 {'reversed': {}, 'new': {}}
Pyr/NO3 {'reversed': {}, 'new': {}}
Pyr/SO4 {'reversed': {}, 'new': {}}
Pyr/SO3 {'reversed': {}, 'new': {}}
H2/CO2 {'reversed': {}, 'new': {}}
H2/O2 {'reversed': {}, 'new': {}}
empty {'reversed': {}, 'new': {}}
Light {'reversed': {}, 'new': {}}


In [24]:
media = model_medias['Ac']

In [27]:
media.get_media_constraints()

{'cpd00001_e0': (-1000, 1000),
 'cpd00067_e0': (-1000, 1000),
 'cpd00009_e0': (-1000, 1000),
 'cpd00011_e0': (-1000, 1000)}

In [28]:
atp_correction.msgapfill.run_gapfilling(media, 'bio2')

/usr/local/lib/python3.8/site-packages/cobra/util/solver.py:508 UserWarning: Solver status is 'infeasible'.


{'reversed': {}, 'new': {}}

In [54]:
import cobrakbase
kbase = cobrakbase.KBaseAPI()
template4 = kbase.get_from_ws('CoreArchaeaTemplateMPA_FIX', 82183)

cobrakbase 0.2.8


In [58]:
atp_correction = MSATPCorrection(model, template4, list(model_medias.values()))

In [59]:
atp_correction.disable_noncore_reactions()

In [60]:
atp_correction.evaluate_growth_media()

DEBUG:modelseedpy.core.msatpcorrection:evaluate media <modelseedpy.core.msmedia.MSMedia object at 0x7f76c03ecfd0>
DEBUG:modelseedpy.core.msatpcorrection:evaluate media <modelseedpy.core.msmedia.MSMedia object at 0x7f76c03ecfd0> - 0.551651 (optimal)
DEBUG:modelseedpy.core.msatpcorrection:evaluate media <modelseedpy.core.msmedia.MSMedia object at 0x7f7675de7f40>
DEBUG:modelseedpy.core.msatpcorrection:evaluate media <modelseedpy.core.msmedia.MSMedia object at 0x7f7675de7f40> - 0.000000 (optimal)
DEBUG:modelseedpy.core.msgapfill:gapfill solution objective value 0.668225 (optimal) for media <modelseedpy.core.msmedia.MSMedia object at 0x7f7675de7f40>
DEBUG:modelseedpy.core.msatpcorrection:evaluate media <modelseedpy.core.msmedia.MSMedia object at 0x7f7675de7520>
DEBUG:modelseedpy.core.msatpcorrection:evaluate media <modelseedpy.core.msmedia.MSMedia object at 0x7f7675de7520> - 0.000000 (optimal)
DEBUG:modelseedpy.core.msgapfill:gapfill solution objective value 0.668225 (optimal) for media <mo

In [61]:
for media in atp_correction.media_gapfill_stats:
    print(media.id, atp_correction.media_gapfill_stats[media])

Glc/O2 None
Ac/O2 {'reversed': {}, 'new': {'rxn05893_c0': '>', 'rxn05466_c0': '>', 'EX_cpd00013_e0': '<'}}
Etho/O2 {'reversed': {}, 'new': {'rxn05893_c0': '>', 'rxn05466_c0': '>', 'EX_cpd00013_e0': '<'}}
Pyr/O2 {'reversed': {}, 'new': {'rxn05469_c0': '>', 'EX_cpd00020_e0': '<'}}
Glyc/O2 None
Fum/O2 None
Succ/O2 {'reversed': {}, 'new': {'rxn09269_c0': '>', 'EX_cpd00036_e0': '<'}}
Akg/O2 {'reversed': {}, 'new': {'rxn05493_c0': '>', 'EX_cpd00024_e0': '<'}}
LLac/O2 {'reversed': {}, 'new': {'rxn01057_c0': '>'}}
Dlac/O2 {'reversed': {}, 'new': {'rxn05893_c0': '>', 'rxn05466_c0': '>', 'EX_cpd00013_e0': '<'}}
For/O2 {'reversed': {}, 'new': {'rxn05893_c0': '>', 'rxn05466_c0': '>', 'EX_cpd00013_e0': '<'}}
Glc {'reversed': {}, 'new': {'rxn01106_c0': '<', 'rxn01057_c0': '<'}}
Ac {'reversed': {}, 'new': {'rxn07191_c0': '<', 'rxn05893_c0': '>'}}
Etho {'reversed': {}, 'new': {'rxn07191_c0': '<', 'rxn05893_c0': '>'}}
Pyr {'reversed': {}, 'new': {'rxn07191_c0': '<', 'rxn05893_c0': '>'}}
Glyc {'reversed

In [63]:
atp_correction.noncore_reactions

[]

In [65]:
atp_correction.gapfilling_delta = 1

In [74]:
def determine_growth_media(ATP, max_gapfilling=None):
    """
    Decides which of the test media to use as growth conditions for this model
    :return:
    """
    score_function = lambda media: len(ATP.media_gapfill_stats[media]["new"].keys()) + 0.5*len(ATP.media_gapfill_stats[media]["reversed"].keys())
    ATP.selected_media = []
    media_scores = dict((media, score_function(media)) for media in ATP.media_gapfill_stats if ATP.media_gapfill_stats[media])
    best_score = min(media_scores.values())
    if max_gapfilling is None:
        max_gapfilling = best_score
    for media in media_scores:
        score = media_scores[media]
        print(score, best_score, max_gapfilling, best_score + ATP.gapfilling_delta)
        if score <= max_gapfilling and score <= (best_score + ATP.gapfilling_delta):
            ATP.selected_media.append(media)
atp_correction.gapfilling_delta = 10
determine_growth_media(atp_correction, 10)
len(atp_correction.selected_media)

3.0 1.0 10 11.0
3.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
1.0 1.0 10 11.0
3.0 1.0 10 11.0
3.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
9.0 1.0 10 11.0
1.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
3.0 1.0 10 11.0
3.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
3.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0
3.0 1.0 10 11.0
2.0 1.0 10 11.0
2.0 1.0 10 11.0


31

In [15]:
atp_correction.determine_growth_media()

In [16]:
atp_correction.selected_media

In [25]:
def evaluate_growth_media(ATP):
    ATP.media_gapfill_stats = {}
    ATP.msgapfill.default_gapfill_templates = [ATP.coretemplate]
    if ATP.lp_filename:
        ATP.msgapfill.lp_filename = ATP.lp_filename
    with ATP.model:
        ATP.model.objective = ATP.atp_objective
        pkgmgr = MSPackageManager.get_pkg_mgr(ATP.model)
        for media in ATP.atp_medias:
            pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
            solution = ATP.model.optimize()
            print(solution)
            ATP.media_gapfill_stats[media] = None
            if solution.objective_value == 0:
                ATP.media_gapfill_stats[media] = ATP.msgapfill.run_gapfilling(media, ATP.atp_objective)
evaluate_growth_media(atp_correction)

<Solution 0.552 at 0x7fe99fb67220>
<Solution 0.000 at 0x7fe99f8ae760>


/usr/local/lib/python3.8/site-packages/cobra/util/solver.py:508 UserWarning: Solver status is 'infeasible'.


<Solution 0.000 at 0x7fe99f857dc0>
<Solution 0.000 at 0x7fe99f5da310>
<Solution 0.616 at 0x7fe99ef9ee80>
<Solution 0.317 at 0x7fe99f78aa00>
<Solution 0.000 at 0x7fe99ef9eeb0>
<Solution 0.000 at 0x7fe99f3a45e0>
<Solution 0.000 at 0x7fe99edf5df0>
<Solution 0.000 at 0x7fe99ece7a90>
<Solution 0.000 at 0x7fe99ebd8400>
<Solution 0.000 at 0x7fe99ead3160>
<Solution 0.000 at 0x7fe99e9cda30>
<Solution 0.000 at 0x7fe99e89c310>
<Solution 0.000 at 0x7fe99e8a8580>
<Solution 0.000 at 0x7fe99e629be0>
<Solution 0.000 at 0x7fe99e5360d0>
<Solution 0.000 at 0x7fe99e3e3820>
<Solution 0.000 at 0x7fe99e2b61f0>
<Solution 0.000 at 0x7fe99e188b50>
<Solution 0.000 at 0x7fe99dfd9be0>
<Solution 0.000 at 0x7fe99dead550>
<Solution 0.000 at 0x7fe99dd80370>
<Solution 0.000 at 0x7fe99dbd1af0>
<Solution 0.000 at 0x7fe99daa5430>
<Solution 0.000 at 0x7fe99d971b20>
<Solution 0.000 at 0x7fe99d83aaf0>
<Solution 0.000 at 0x7fe99d68b6a0>
<Solution 0.000 at 0x7fe99d55b310>
<Solution 0.000 at 0x7fe99d421eb0>
<Solution 0.000 at 0

In [13]:
atp_correction.run_atp_correction()

/usr/local/lib/python3.8/site-packages/cobra/util/solver.py:508 UserWarning: Solver status is 'infeasible'.


TypeError: 'Model' object is not subscriptable

In [61]:
atp_correction.disable_noncore_reactions()

In [63]:
from modelseedpy.fbapkg.mspackagemanager import MSPackageManager
def evaluate_growth_media2(ATPM):
    ATPM.media_gapfill_stats = {}
    ATPM.msgapfill.default_gapfill_templates = [ATPM.coretemplate]
    if ATPM.lp_filename:
        ATPM.msgapfill.lp_filename = ATPM.lp_filename
    with ATPM.model:
        ATPM.model.objective = ATPM.atp_objective
        pkgmgr = MSPackageManager.get_pkg_mgr(ATPM.model)
        for media in ATPM.atp_medias:
            print(media.id)
            pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
            solution = ATPM.model.optimize()
            print(media.id, solution)
            ATPM.media_gapfill_stats[media] = None
            if solution.objective_value == 0:
                ATPM.media_gapfill_stats[media] = ATPM.msgapfill.run_gapfilling(media, ATPM.atp_objective)
evaluate_growth_media2(atp_correction)

Glc/O2
Glc/O2 <Solution 0.552 at 0x7f497f5ab1f0>
Ac/O2
Ac/O2 <Solution 0.000 at 0x7f497e45d730>


/usr/local/lib/python3.8/site-packages/cobra/util/solver.py:508 UserWarning: Solver status is 'infeasible'.


Etho/O2
Etho/O2 <Solution 0.000 at 0x7f497e309460>
Pyr/O2
Pyr/O2 <Solution 0.000 at 0x7f497e1dbbe0>
Glyc/O2
Glyc/O2 <Solution 0.616 at 0x7f497e02e790>
Fum/O2
Fum/O2 <Solution 0.317 at 0x7f497e31c7c0>
Succ/O2
Succ/O2 <Solution 0.000 at 0x7f497e02e5e0>
Akg/O2
Akg/O2 <Solution 0.000 at 0x7f497df01460>
LLac/O2
LLac/O2 <Solution 0.000 at 0x7f497ddda070>
Dlac/O2
Dlac/O2 <Solution 0.000 at 0x7f497dc2bd30>
For/O2
For/O2 <Solution 0.000 at 0x7f497dafc460>
Glc
Glc <Solution 0.000 at 0x7f497d9ccac0>
Ac
Ac <Solution 0.000 at 0x7f497d896b50>
Etho
Etho <Solution 0.000 at 0x7f497d75f700>
Pyr
Pyr <Solution 0.000 at 0x7f497d5b0520>
Glyc
Glyc <Solution 0.000 at 0x7f497d47ba30>
Fum
Fum <Solution 0.000 at 0x7f497d34e5b0>
Succ
Succ <Solution 0.000 at 0x7f497d214af0>
Akg
Akg <Solution 0.000 at 0x7f497d0e0b80>
For/NO2
For/NO2 <Solution 0.000 at 0x7f497cf301c0>
For/NO3
For/NO3 <Solution 0.000 at 0x7f497cdf9fa0>
Glc/DMSO
Glc/DMSO <Solution 0.000 at 0x7f497cccbb80>
Glc/TMAO
Glc/TMAO <Solution 0.000 at 0x7f497cb

In [66]:
atp_correction.evaluate_growth_media()

In [57]:
atp_correction.determine_growth_media()

In [59]:
for media in atp_correction.selected_media:
    print(media.id)

Glc/O2
Ac/O2
Etho/O2
Pyr/O2
Glyc/O2
Fum/O2
Succ/O2
Akg/O2
Dlac/O2
For/O2
Ac
Etho
Pyr
Succ
Akg
For/NO2
For/NO3
Pyr/DMSO
Pyr/TMAO
Pyr/NO
Pyr/NO2
Pyr/NO3
Pyr/SO4
Pyr/SO3
H2/CO2
H2/O2
empty
Light


In [1]:
import modelseedpy

modelseedpy 0.2.2


In [2]:
%run /home/fliu/workspace/python3/ModelSEEDpy/tests/core/test_msatpcorreption.py

In [11]:
def get_model(ko):
    def _method(ko=ko, added_compounds=None, added_reactions=None):
        if ko is None:
            ko = []
        with open(
            '/home/fliu/workspace/python3/ModelSEEDpy/tests/test_data/e_coli_core.json',
            "r",
        ) as fh:
            model_json = json.load(fh)
            model_json["compartments"] = {
                k + "0": v for (k, v) in model_json["compartments"].items()
            }
            metabolites = {}
            for m in model_json["metabolites"]:
                m["id"] += "0"
                m["compartment"] += "0"
                metabolites[m["id"]] = m
            for r in model_json["reactions"]:
                r["metabolites"] = {i + "0": v for (i, v) in r["metabolites"].items()}
                compartments = set(
                    [metabolites[k]["compartment"] for k in r["metabolites"].keys()]
                )
                if r["id"].endswith("_e"):
                    r["id"] += "0"
                elif len(compartments) == 1:
                    r["id"] += "_" + list(compartments)[0]
                else:
                    r["id"] += (
                        "_" + "c0"
                    )  # hack cause there is only combo between e0 and c0

            model_json["reactions"] = [
                x for x in model_json["reactions"] if x["id"] not in ko
            ]

            if added_compounds:
                for o in added_compounds:
                    model_json["metabolites"].append(o)
            if added_reactions:
                for o in added_reactions:
                    model_json["reactions"].append(o)
            model = cobra.io.from_json(json.dumps(model_json))
            model.reactions.ATPM_c0.lower_bound = 0
            model.reactions.ATPM_c0.upper_bound = 1000
            return model

    return _method(ko)

In [45]:
import logging
logger = logging.getLogger(__name__)

In [96]:
model = get_model(["NADH16_c0", "CYTBD_c0", "O2t_c0", "GLCpts_c0"])
with open('/home/fliu/workspace/python3/ModelSEEDpy/tests/test_data/template_core_bigg.json', 'r') as fh:
    template = MSTemplateBuilder.from_dict(json.load(fh)).build()
media_glucose_aerobic = MSMedia.from_dict(
        {
            "glc__D": (-1, 1000),
            "o2": (-1000, 1000),
            "h": (-1000, 1000),
            "h2o": (-1000, 1000),
        }
    )
media_glucose_aerobic.id = 'glc/o2'
media_acetate_aerobic = MSMedia.from_dict(
        {
            "ac": (-1, 1000),
            "o2": (-1000, 1000),
            "h": (-1000, 1000),
            "h2o": (-1000, 1000),
        }
    )
media_acetate_aerobic.id = 'ac/o2'
medias = [media_glucose_aerobic, media_acetate_aerobic]

In [97]:
%run /home/fliu/workspace/python3/ModelSEEDpy/modelseedpy/core/msatpcorrection.py
atp_correction = MSATPCorrection(
        model,
        template,
        medias,
        atp_hydrolysis_id="ATPM_c0",
        load_default_medias=False,
    )

In [98]:
atp_correction.atp_medias

[[<modelseedpy.core.msmedia.MSMedia at 0x7ff2586535b0>, 0.01],
 [<modelseedpy.core.msmedia.MSMedia at 0x7ff2572e4b20>, 0.01]]

In [99]:
model

Name,e_coli_core
Memory address,7ff258653370
Number of metabolites,72
Number of reactions,91
Number of genes,137
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM_c0 - 1.0*BIOMASS_Ecoli_core_w_GAM_c0_reverse_70c47
Compartments,"extracellular space, cytosol"


In [100]:
atp_correction.evaluate_growth_media()

{'glc/o2': 0.0, 'ac/o2': 0.0}

In [101]:
atp_correction.media_gapfill_stats

{<modelseedpy.core.msmedia.MSMedia at 0x7ff2586535b0>: {'reversed': {},
  'new': {'GLCpts_c0': '>'},
  'media': <modelseedpy.core.msmedia.MSMedia at 0x7ff2586535b0>,
  'target': 'ATPM_c0',
  'minobjective': 0.01,
  'binary_check': False},
 <modelseedpy.core.msmedia.MSMedia at 0x7ff2572e4b20>: {'reversed': {},
  'new': {'CYTBD_c0': '>', 'NADH16_c0': '>', 'O2t_c0': '>'},
  'media': <modelseedpy.core.msmedia.MSMedia at 0x7ff2572e4b20>,
  'target': 'ATPM_c0',
  'minobjective': 0.01,
  'binary_check': False}}

In [102]:
atp_correction.determine_growth_media()

In [105]:
atp_correction.selected_media[0].id

'glc/o2'

In [106]:
atp_correction.apply_growth_media_gapfilling()

In [107]:
media_eval = atp_correction.evaluate_growth_media()

In [108]:
media_eval

{'glc/o2': 2.75, 'ac/o2': 0.0}

In [109]:
atp_correction.expand_model_to_genome_scale()

In [110]:
tests = atp_correction.build_tests()

In [113]:
for t in tests:
    print(t['media'].id, t)

empty {'media': <modelseedpy.core.msmedia.MSMedia object at 0x7ff2572ed850>, 'is_max_threshold': True, 'threshold': 1e-05, 'objective': 'ATPM_c0'}
glc/o2 {'media': <modelseedpy.core.msmedia.MSMedia object at 0x7ff2586535b0>, 'is_max_threshold': True, 'threshold': 3.3, 'objective': 'ATPM_c0'}


In [114]:
tests

[{'media': <modelseedpy.core.msmedia.MSMedia at 0x7ff2572ed850>,
  'is_max_threshold': True,
  'threshold': 1e-05,
  'objective': 'ATPM_c0'},
 {'media': <modelseedpy.core.msmedia.MSMedia at 0x7ff2586535b0>,
  'is_max_threshold': True,
  'threshold': 3.3,
  'objective': 'ATPM_c0'}]